# =Programming Sample=

## CNN classifier for the MNIST dataset

### Intenstions

In this notebook, we will write code to build, compile and fit a convolutional neural network (CNN) model to the MNIST dataset of images of handwritten digits.


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D,  Softmax, MaxPooling2D
from tensorflow.keras.preprocessing import image

![MNIST overview image](data/mnist.png)

#### The MNIST dataset

[MNIST dataset](http://yann.lecun.com/exdb/mnist/) consists of a training set of 60,000 handwritten digits with corresponding labels, and a test set of 10,000 images. The images have been normalised and centred. The dataset is frequently used in machine learning research, and has become a standard benchmark for image classification models. 

- Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. "Gradient-based learning applied to document recognition." Proceedings of the IEEE, 86(11):2278-2324, November 1998.

We aim to construct a neural network that classifies images of handwritten digits into one of 10 classes.

### Load and preprocess the data

In [ ]:
mnist_data = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_data.load_data()

First, preprocess the data by scaling the training and test images so their values lie in the range from 0 to 1.

In [ ]:
def scale_mnist_data(train_images, test_images):
    """
    m=0
    for i in train_images:
        for j in i:
            for k in j:
                if m < k: m=k
    #print(m)
    if m > 1.00:
        train_images = train_images / float(m) #255.0
        test_images = test_images / float(m) #255.0
    """
    ###FASTER TO CODE:
    return  train_images / 255.0,  test_images / 255.0

In [ ]:
scaled_train_images, scaled_test_images = scale_mnist_data(train_images, test_images)

In [ ]:
# Add a dummy channel dimension

scaled_train_images = scaled_train_images[..., np.newaxis]
scaled_test_images = scaled_test_images[..., np.newaxis]

### Build the convolutional neural network model

We are now ready to construct a model to fit to the data:

* The model should use the `input_shape` in the function argument to set the input size in the first layer.
* A 2D convolutional layer with a 3x3 kernel and 8 filters. Use 'SAME' zero padding and ReLU activation functions. Make sure to provide the `input_shape` keyword argument in this first layer.
* A max pooling layer, with a 2x2 window, and default strides.
* A flatten layer, which unrolls the input into a one-dimensional tensor.
* Two dense hidden layers, each with 64 units and ReLU activation functions.
* A dense output layer with 10 units and the softmax activation function.

In [ ]:
def get_model(input_shape):
    model = Sequential ()
    model.add(Conv2D(8, (3,3), activation='relu', padding='SAME', input_shape=input_shape))
    model.add(MaxPooling2D(3,3))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    return model

In [ ]:
model = get_model(scaled_train_images[0].shape)

### Compile the model

In [ ]:
def compile_model(model):
    model.compile(
        optimizer='adam', 
        loss='sparse_categorical_crossentropy', 
        ##categorical_crossentropy 
        ##sparse_categorical_crossentropy
        metrics=['accuracy']
)

In [ ]:
compile_model(model)

### Fit the model to the training data

In [ ]:
def train_model(model, scaled_train_images, train_labels):
    hisptory = model.fit(scaled_train_images, train_labels, epochs=5 )
    return hisptory
    

In [ ]:
history = train_model(model, scaled_train_images, train_labels)

#### Plot the learning curves

We will now plot two graphs:
- Epoch vs accuracy
- Epoch vs loss

We will load the model history into a pandas `DataFrame` and use the `plot` method to output the required graphs.

In [ ]:
frame = pd.DataFrame(history.history)

In [ ]:
acc_plot = frame.plot(y="accuracy", title="Accuracy vs Epochs", legend=False)
acc_plot.set(xlabel="Epochs", ylabel="Accuracy")

In [ ]:
acc_plot = frame.plot(y="loss", title = "Loss vs Epochs",legend=False)
acc_plot.set(xlabel="Epochs", ylabel="Loss")

### Evaluate the model

In [ ]:
def evaluate_model(model, scaled_test_images, test_labels):
    r = model.evaluate(scaled_test_images, test_labels, verbose=2)
    return r

In [ ]:
test_loss, test_accuracy = evaluate_model(model, scaled_test_images, test_labels)
print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_accuracy}")

### Model predictions

In [ ]:
num_test_images = scaled_test_images.shape[0]

random_inx = np.random.choice(num_test_images, 4)
random_test_images = scaled_test_images[random_inx, ...]
random_test_labels = test_labels[random_inx, ...]

predictions = model.predict(random_test_images)

fig, axes = plt.subplots(4, 2, figsize=(16, 12))
fig.subplots_adjust(hspace=0.4, wspace=-0.2)

for i, (prediction, image, label) in enumerate(zip(predictions, random_test_images, random_test_labels)):
    axes[i, 0].imshow(np.squeeze(image))
    axes[i, 0].get_xaxis().set_visible(False)
    axes[i, 0].get_yaxis().set_visible(False)
    axes[i, 0].text(10., -1.5, f'Digit {label}')
    axes[i, 1].bar(np.arange(len(prediction)), prediction)
    axes[i, 1].set_xticks(np.arange(len(prediction)))
    axes[i, 1].set_title(f"Categorical distribution. Model prediction: {np.argmax(prediction)}")
    
plt.show()